# Домашняя работа №4

# Выполнил: Лобанов Александр

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import shutil
import re
import requests
import pandas as pd
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import DenseNet169, VGG16, Xception, ResNet152V2
import tensorflow.image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))
    
    ax[0].plot(epochs, acc, 'bo', label='Точность обучения')
    ax[0].plot(epochs, val_acc, 'b', label='Точность валидации')
    ax[0].set_title('Точность (accuracy)')
    ax[0].legend()
    ax[0].grid()

    ax[1].plot(epochs, loss, 'bo', label='Функция потерь обучения')
    ax[1].plot(epochs, val_loss, 'b', label='Функция потерь валидации')
    ax[1].set_title('Функция потерь')
    ax[1].legend()
    ax[1].grid()

In [ ]:
import zipfile

with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip","r") as z:
    z.extractall(".") 

In [ ]:
# Retrieving a list of directories in each folder

DIR_TRAIN = "/kaggle/working/train/"
DIR_TEST = "/kaggle/working/test1"

train_imgs = os.listdir(DIR_TRAIN)
test_imgs = os.listdir(DIR_TEST)

In [ ]:
image = load_img("/kaggle/working/train/"+ "dog.10008.jpg")
plt.imshow(image)
plt.axis("off")
plt.show()

In [ ]:
# Creating a DataFrame for our train set

category = [x.split(".")[0] for x in train_imgs]
df = pd.DataFrame({"Filename":train_imgs, "Category":category})
df.head()

In [ ]:
df_train, df_test = train_test_split(df, test_size = 7500, 
                                     stratify=df['Category'],
                                     random_state=42)

In [ ]:
df_train['Category'].value_counts()

In [ ]:
df_test['Category'].value_counts()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
        df_train,
        directory = DIR_TRAIN,
        x_col = 'Filename',
        y_col = 'Category',
        target_size = (200, 200),
        class_mode = 'binary',
        batch_size = 20)

test_generator = test_datagen.flow_from_dataframe(
        df_test,
        directory = DIR_TRAIN,
        x_col = 'Filename',
        y_col = 'Category',
        target_size = (200, 200),
        class_mode = 'binary',
        batch_size = 20)



In [ ]:
model = Sequential()
model.add(Conv2D(8, (3, 3), padding='SAME', activation='relu',
                        input_shape=(200, 200, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(16, (3, 3), padding='SAME', activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=2e-4),
              metrics=['acc'])

In [ ]:
history_baseline = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=test_generator,
      validation_steps=50)

In [ ]:
plot_history(history_baseline)

### Обучение модели с применением аугментации

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
        df_train,
        directory = DIR_TRAIN,
        x_col = 'Filename',
        y_col = 'Category',
        target_size = (200, 200),
        class_mode = 'binary',
        batch_size = 20)

test_generator = test_datagen.flow_from_dataframe(
        df_test,
        directory = DIR_TRAIN,
        x_col = 'Filename',
        y_col = 'Category',
        target_size = (200, 200),
        class_mode = 'binary',
        batch_size = 20)

In [ ]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(8, (3, 3), padding='SAME', activation='relu',
                        input_shape=(200, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(16, (3, 3), padding='SAME', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(learning_rate=2e-4),
              metrics=['acc'])

In [ ]:
history_augmentation = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=140,
      validation_data=test_generator,
      validation_steps=50)

In [ ]:
plot_history(history_augmentation)

### Заключение

Сравнивая значения accuracy на тестовой выборке базовой модели (history_baseline) и на модифицированной модели (history_augmentation), можно сделать вывод, что оптимизация при помощи приема аугментации проведена успешна. 

> Значения accuracy значительно улучшилось: 
* Accuracy на history_baseline: 0.7730
* Accuracy на history_augmentation: 0.8840